# Przetwarzanie języka naturalnego – lab6
## Mateusz Kocot

In [60]:
from datasets import load_dataset

import numpy as np
from sklearn.naive_bayes import GaussianNB

In [61]:
dataset_1 = load_dataset('poleval2019_cyberbullying', 'task01')
dataset_2 = load_dataset('poleval2019_cyberbullying', 'task02')

Found cached dataset poleval2019_cyberbullying (C:/Users/MatiX/.cache/huggingface/datasets/poleval2019_cyberbullying/task01/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 250.33it/s]
Found cached dataset poleval2019_cyberbullying (C:/Users/MatiX/.cache/huggingface/datasets/poleval2019_cyberbullying/task02/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 250.39it/s]


In [62]:
x_train_1, y_train_1 = dataset_1['train']['text'], dataset_1['train']['label']
x_test_1, y_test_1 = dataset_1['test']['text'], dataset_1['test']['label']

len(x_train_1), len(y_train_1), len(x_test_1), len(y_test_1)

(10041, 10041, 1000, 1000)

In [63]:
x_train_2, y_train_2 = dataset_2['train']['text'], dataset_2['train']['label']
x_test_2, y_test_2 = dataset_2['test']['text'], dataset_2['test']['label']

len(x_train_2), len(y_train_2), len(x_test_2), len(y_test_2)

(10041, 10041, 1000, 1000)

# Task 2

In [57]:
gnb = GaussianNB()
gnb.fit(x_train_1, y_train_1)

ValueError: dtype='numeric' is not compatible with arrays of bytes/strings.Convert your data to numeric values explicitly instead.